In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import json
from common_module import *
from plotting_module import *
import warnings

In [2]:
%matplotlib qt

set_matplotlib_defaults()

warnings.filterwarnings('ignore')

In [3]:
# Load trained network
N_p_free = tf.keras.models.load_model("../data/N_P_free/Weights", compile=False)
N_t_free = tf.keras.models.load_model("../data/N_T_free/Weights", compile=False)

N_p_fixed = tf.keras.models.load_model("../data/N_P_fixed/Weights", compile=False)
N_t_fixed = tf.keras.models.load_model("../data/N_T_fixed/Weights", compile=False)

# Load parameters
with open('../parameters.json', 'r') as f:
   parameters = json.load(f)['parameters']

parameters

{'R_lc': 16.67,
 'r_c': 16.5033,
 'Pc_mode': 'free',
 'Pc_fixed': 0.075284943,
 'coordinates': 'spherical',
 'n_1': 500,
 'n_2': 500,
 'extent': 3}

In [4]:
# Grid
r, theta, q, mu, rho, z = get_grid(parameters)

if parameters['coordinates'] == 'spherical':

    x1, x2 = r, theta
    X, Z = r*np.sin(theta), r*np.cos(theta)

elif parameters['coordinates'] == 'compactified':

    x1, x2 = q, mu
    X, Z = np.sqrt(1 - mu **2) * q, mu * q

elif parameters['coordinates'] == 'cylindrical':

    x1, x2 = rho, z
    X, Z = rho, z

# PINN input
X_test = generate_test (x1, x2, coordinates=parameters['coordinates'])

In [5]:
P_free, Pc_free, T_free, Tprime_free = get_scalar_functions(X_test, N_p_free, N_t_free, {**parameters, 'Pc_mode':'free'})
P_fixed, Pc_fixed, T_fixed, Tprime_fixed = get_scalar_functions(X_test, N_p_fixed, N_t_fixed, {**parameters, 'Pc_mode':'fixed'})

In [6]:
B_1_free, B_2_free, B_3_free, \
E_1_free, E_2_free, E_3_free, \
B_mag_free, E_mag_free, B_pol_free, \
E_dot_B_free, div_B_free, div_E_free = get_fields (x1, x2, P_free, T_free, parameters)

B_1_fixed, B_2_fixed, B_3_fixed, \
E_1_fixed, E_2_fixed, E_3_fixed, \
B_mag_fixed, E_mag_fixed, B_pol_fixed, \
E_dot_B_fixed, div_B_fixed, div_E_fixed = get_fields (x1, x2, P_fixed, T_fixed, parameters)

In [7]:
def get_pinn (X, P, Pc, N_p, N_t, parameters):
   
   Np = N_p (X)[:,0].numpy().reshape([parameters['n_1'], parameters['n_2']])
   Nt = N_t (tf.convert_to_tensor(P.ravel())).numpy().reshape([parameters['n_1'], parameters['n_2']])

   fb = f_b (X, Pc, parameters['r_c']).numpy().reshape([parameters['n_1'], parameters['n_2']])
   hb = h_b (X, parameters['r_c']).numpy().reshape([parameters['n_1'], parameters['n_2']])

   return Np, Nt, fb, hb

Np_free, Nt_free, fb_free, hb_free = get_pinn (X_test, P_free, Pc_free, N_p_free, N_t_free, {**parameters, 'Pc_mode':'free'})
Np_fixed, Nt_fixed, fb_fixed, hb_fixed = get_pinn (X_test, P_fixed, Pc_fixed, N_p_fixed, N_t_fixed, {**parameters, 'Pc_mode':'fixed'})

In [8]:
# plot_pinn (X, Z, P_free, Pc_free, fb_free, hb_free, Np_free, parameters)
# plot_pinn (X, Z, P_fixed, Pc_fixed, fb_fixed, hb_fixed, Np_fixed, parameters)

In [9]:
# plot_scalar_functions(X, Z, P_free, Pc_free, T_free, Tprime_free, parameters)
# plot_scalar_functions(X, Z, P_fixed, Pc_fixed, T_fixed, Tprime_fixed, parameters)

In [10]:
# plot_fields (X, Z, P_free, Pc_free, B_mag_free, E_mag_free, div_E_free, parameters)
# plot_fields (X, Z, P_fixed, Pc_fixed, B_mag_fixed, E_mag_fixed, div_E_fixed, parameters)

In [26]:
def plot_T_of_P (P, Pc, T, T_prime, parameters):
    
   fig, axes = plt.subplots(1, 3)

   # Mask T_prime to hide divergent values due to numerical derivation
   mask = np.abs(T_prime) < 0.15
   T_prime_masked = np.where(mask, T_prime, np.nan)
   
   #Calculate monopole solution for normalisation and comparison
   # P_mon = 1 / parameters['R_lc'] (1 - )
   scalars = [-T, -T_prime_masked, T * T_prime_masked]
   # scalars_monopole = [-T, -T_prime_masked, T * T_prime_masked]

   for i, ax in enumerate(axes):
      f = scalars[i].ravel()

      ax.set_xlim([0, 1.1 * Pc])
      ax.axvline(Pc, c='k', alpha=0.5, ls='--')

      ax.scatter(P.ravel(), f, s=10, marker='*')

   return

In [27]:
plot_T_of_P (P_free, Pc_free, T_free, Tprime_free, parameters)
plot_T_of_P (P_fixed, Pc_fixed, T_fixed, Tprime_fixed, parameters)

In [13]:
x = np.arange(10).reshape((2,5))
mask = x%2 == 0
x_masked = np.where(mask, x, 0)
x_masked

array([[0, 0, 2, 0, 4],
       [0, 6, 0, 8, 0]])